In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "API KEY HERE"

In [ ]:
!pip install langchain langchain-openai langchain_core langchain_community

In [20]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Dictionary to hold chat histories by session_id
session_chains = {}

def setup_chat(model: str = "gpt-3.5-turbo-1106", session_id: str = "default"):
    chat = ChatOpenAI(model=model)

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful assistant. Answer all questions to the best of your ability."),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
        ]
    )

    chain = prompt | chat

    # Set up unique chat history for this session
    history = ChatMessageHistory()

    chain_with_history = RunnableWithMessageHistory(
        chain,
        lambda _: history,
        input_messages_key="input",
        history_messages_key="chat_history",
    )

    # Store in dictionary
    session_chains[session_id] = {
        "chain": chain_with_history,
        "history": history,
    }

def chat_with_session(text: str, session_id: str = "default"):
    if session_id not in session_chains:
        raise ValueError(f"No chat session found with ID '{session_id}'. Please run setup_chat() first.")

    chain = session_chains[session_id]["chain"]

    response = chain.invoke(
        {"input": text},
        {"configurable": {"session_id": session_id}},
    )

    return response.content


In [23]:
# Possible models : "gpt-4o" "o3-mini-2025-01-31"

# Setup once
setup_chat(model="gpt-4o", session_id="user123")

# Call multiple times while keeping history
print(chat_with_session("Hello!", session_id="user123"))


Hello! How can I assist you today?


In [24]:
print(chat_with_session("What did I just say?", session_id="user123"))

You just said "Hello!" and I responded to you. If there's anything specific you'd like to discuss or any questions you have, feel free to let me know!
